<a href="https://colab.research.google.com/github/ChloeMorgana/Dissertation-Project/blob/main/PubMedBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loading of DrugProt dataset

Downloading and unzipping file

In [1]:
!wget https://zenodo.org/record/5042151/files/drugprot-gs-training-development.zip
!unzip drugprot-gs-training-development.zip

--2022-11-28 09:16:21--  https://zenodo.org/record/5042151/files/drugprot-gs-training-development.zip
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3796908 (3.6M) [application/octet-stream]
Saving to: ‘drugprot-gs-training-development.zip’

drugprot-gs-trainin 100%[===================>]   3.62M   407KB/s    in 9.3s    

2022-11-28 09:16:32 (399 KB/s) - ‘drugprot-gs-training-development.zip’ saved [3796908/3796908]

Archive:  drugprot-gs-training-development.zip
   creating: drugprot-gs-training-development/
  inflating: drugprot-gs-training-development/README.pdf  
   creating: drugprot-gs-training-development/development/
  inflating: drugprot-gs-training-development/development/drugprot_development_relations.tsv  
  inflating: drugprot-gs-training-development/development/drugprot_development_entities.tsv  
  inflating: drugprot-gs-training-development/d

Combining all three files together.

In [2]:
def loadDrugProt(abstracts_filename, entities_filename, relations_filename):

  docs = {}

  # Load the title/abstracts text in as documents
  with open(abstracts_filename, encoding='utf8') as f:
    for lineno,line in enumerate(f):
      split = line.strip('\n').split('\t')
      assert len(split) == 3, f"Expected 3 columns but got {len(split)} on line {lineno+1}"
      pubmed_id, title, abstract = split
      pubmed_id = int(pubmed_id)

      combined_text = title + '\n' + abstract
      docs[pubmed_id] = {'pubmed_id':pubmed_id, 'text':combined_text, 'entities':{}, 'relations':[]}

  # Load the entities and match them up with the documents
  with open(entities_filename, encoding='utf8') as f:
    for lineno,line in enumerate(f):
      split = line.strip('\n').split('\t')
      assert len(split) == 6, f"Expected 6 columns but got {len(split)} on line {lineno+1}"
      pubmed_id, entity_id, entity_type, start_coord, end_coord, entity_text = split

      pubmed_id = int(pubmed_id)
      start_coord = int(start_coord)
      end_coord = int(end_coord)

      assert pubmed_id in docs, f"Did not find matching document for pubmed_id={pubmed_id}"
      doc = docs[pubmed_id]

      assert doc['text'][start_coord:end_coord] == entity_text, f"Text for entity with coords {start_coord}:{end_coord} in document (pubmed_id={pubmed_id} does not match expected. 'f{doc['text'][start_coord:end_coord]}' != '{entity_text}'"

      entity = {'type':entity_type, 'start':start_coord, 'end':end_coord, 'text':entity_text}
      doc['entities'][entity_id] = entity

  if relations_filename is not None:
    # Load the relations and match them up with the entities in the corresponding document
    with open(relations_filename, encoding='utf8') as f:
      for lineno,line in enumerate(f):
        split = line.strip('\n').split('\t')
        assert len(split) == 4, f"Expected 4 columns but got {len(split)} on line {lineno+1}"

        pubmed_id, relation_type, arg1, arg2 = split

        pubmed_id = int(pubmed_id)
        assert arg1.startswith('Arg1:'), f"Relation argument should start with 'Arg1:'. Got: {arg1}"
        assert arg2.startswith('Arg2:'), f"Relation argument should start with 'Arg2:'. Got: {arg2}"

        # Remove arg1/arg2 from text
        arg1 = arg1.split(':')[1]
        arg2 = arg2.split(':')[1]

        assert pubmed_id in docs, f"Did not find matching document for pubmed_id={pubmed_id}"
        doc = docs[pubmed_id]

        assert arg1 in doc['entities'], f"Couldn't find entity with id={arg1} in document with pubmed_id={pubmed_id}"
        assert arg2 in doc['entities'], f"Couldn't find entity with id={arg2} in document with pubmed_id={pubmed_id}"

        relation = {'type':relation_type,'arg1':arg1,'arg2':arg2}
        doc['relations'].append(relation)

  # Convert the dictionary of documents (indexed by pubmed_id) into a simpler dictionary
  docs = sorted(docs.values(), key=lambda x:x['pubmed_id'])

  return docs

Applying function to dataset

In [3]:
docs = loadDrugProt('drugprot-gs-training-development/training/drugprot_training_abstracs.tsv',
                    'drugprot-gs-training-development/training/drugprot_training_entities.tsv',
                    'drugprot-gs-training-development/training/drugprot_training_relations.tsv')

Observation of doc structure


In [4]:
search = [ d for i,d in enumerate(docs) if d['pubmed_id'] == 1280065]
assert len(search) == 1, "Something went wrong and couldn't find the document we want"

doc = search[0]
doc

{'pubmed_id': 1280065,
 'text': 'Analysis of plasmin binding and urokinase activation of plasminogen bound to the Heymann nephritis autoantigen, gp330.\nPreviously, we demonstrated that the Heymann nephritis autoantigen, gp330, can serve as a receptor site for plasminogen. This binding was not significantly inhibited by the lysine analogue epsilon-amino caproic acid (EACA), indicating that plasminogen binding was not just through lysine binding sites as suggested for other plasminogen binding sites. We now report that once plasminogen is bound to gp330, it can be converted to its active form of plasmin by urokinase. This conversion of plasminogen to plasmin proceeds at a faster rate when plasminogen is first prebound to gp330. Although there is a proportional increase in the Vmax of the urokinase-catalyzed reaction with increasing gp330 concentrations, no change in Km was observed. Once activated, plasmin remains bound to gp330 in an active state capable of cleaving the chromogenic tri

Total number of relations

In [5]:
num = 0
for doc in docs:
  num += len(doc['relations'])

print(num)

17288


# Encoding Step

In [46]:
#!pip install torch
!pip install transformers
!pip install segtok

#import transformers as ppb
from transformers import BertModel, AutoTokenizer
import pandas as pd
import numpy as np
import nltk
import torch
from segtok.segmenter import split_single

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Adding in start and end entity tags to biomedical text, e.g. [E1] CHEMICAL [/E1] and [E2] GENE [/E2]

In [7]:
def isRelation(doc, key):
  relations = doc['relations']
  entities = doc['entities'].keys()
  for r in relations:
    if (key == r['arg1']) or (key == r['arg2']):
      return True
  return False

In [27]:
def addEntityTags(docs):

  for doc in docs:
    doc['labels'] = []
    str = ''
    index = 0
    n = 1
    keys = doc['entities'].keys()
    key_list = sorted(keys, key= lambda x: (doc['entities'][x]['start']))
    for k in key_list:
      if isRelation(doc, k):
        doc['labels'].append(1)
      else:
        doc['labels'].append(0)
      v = doc['entities'][k]
      start = v['start']
      end = v['end']
      name = v['text']
      if v['type'] == 'CHEMICAL':
        str += doc['text'][index:start] + f'[E1] ' + doc['text'][start:end] + f' [/E1]'
      else:
        str += doc['text'][index:start] + f'[E2] ' + doc['text'][start:end] + f' [/E2]'
      index = end
    doc['tagged_text'] = str 

addEntityTags(docs)
print(docs[0]['tagged_text'])
print(len(docs[0]['labels']))
      

Analysis of [E2] plasmin [/E2] binding and [E2] urokinase [/E2] activation of [E2] plasminogen [/E2] bound to the [E2] Heymann nephritis autoantigen [/E2], [E2] gp330 [/E2].
Previously, we demonstrated that the [E2] Heymann nephritis autoantigen [/E2], [E2] gp330 [/E2], can serve as a receptor site for [E2] plasminogen [/E2]. This binding was not significantly inhibited by the [E1] lysine [/E1] analogue [E1] epsilon-amino caproic acid [/E1] ([E1] EACA [/E1]), indicating that [E2] plasminogen [/E2] binding was not just through [E1] lysine [/E1] binding sites as suggested for other [E2] plasminogen [/E2] binding sites. We now report that once [E2] plasminogen [/E2] is bound to [E2] gp330 [/E2], it can be converted to its active form of [E2] plasmin [/E2] by [E2] urokinase [/E2]. This conversion of [E2] plasminogen [/E2] to [E2] plasmin [/E2] proceeds at a faster rate when [E2] plasminogen [/E2] is first prebound to [E2] gp330 [/E2]. Although there is a proportional increase in the Vmax o

Reducing the text only to sentences potentially containing relations between chemicals and genes:

In [29]:
from collections import Counter

def relevantText():

  target_text = []
  labels = []

  for doc in docs:
    txt = doc['tagged_text']
    lbls = doc['labels']

    split_text = []
    split_text += split_single(txt)
    offset = 0

    for s in split_text:
      if '[/E' in s:
        o, l, new_lbls = findLabels(s,lbls,offset)
        target_text.append(s)
        lbls = new_lbls
        offset += o
        labels.append(l)

  return target_text, labels

def findLabels(text, labels, offset=0):
    words = text.split(" ")
    counter = Counter(words)
    idx = counter['[E2]']+counter['[E1]']
    isRel = False
    #print(labels)
    #print(idx)
    for i in range(idx):
      if labels[i]==1:
        isRel = True
    offset+=idx
    if isRel:
      label = 1
    else:
      label = 0
    l = labels[idx:]
    return offset, label, l

target_text, labels= relevantText()
t = target_text[:49]
l = labels[:49]

print(len(target_text))
print(len(labels))
print(l)


29234
29234
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1]


There are around 17288 total relations, therefore there some sentences that contain entities but don't actually contain a relation

In [18]:
print(len(target_text))

29234


# Embedding Step

Loading pre-trained PubMedBERT model + tokenizer

In [47]:
model = BertModel.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext",
                                  output_hidden_states = True,
                                  )

tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext") 

tokenizer.add_tokens(["[E1]", "[/E1]", "[E2]", "[/E2]"])

tokenizer.vocab["[E1]"]

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


30522

Adding more tags and tokenising input

In [48]:
def bert_text_prep(text, tokenizer):

  #CLS lets BERT know when start of sentence begins, SEP indicates start of second sentence.
  marked_text = "[CLS]" + text + "[SEP]"

  #Tokenizes text
  tokenized_text = tokenizer.tokenize(marked_text)

  #Converts to ids
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  segments_ids = [1]*len(indexed_tokens)

  # Converts inputs to tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  return tokenized_text, tokens_tensor, segments_tensors

Converting input into embeddings

In [49]:
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
  with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2][1:]

  token_embeddings = hidden_states[-1]
  #Collapsing tensor into 1 dimension
  token_embeddings = torch.squeeze(token_embeddings, dim=0)
  #converting tensors to lists
  list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

  return list_token_embeddings

Getting CLS embeddings for each sentence



In [50]:
test = target_text[:5]
cls_embeddings = []

for text in t:
    tokenized_text, tokens_tensor, segments_tensors = bert_text_prep(text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
    cls_embeddings.append(list_token_embeddings[0])

print(len(cls_embeddings))

49


## Classification Step

Relevant imports:



In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

Splitting the embeddings into train/test sets

In [52]:
train_features, test_features, train_labels, test_labels = train_test_split(cls_embeddings, l)

Classifying these using logistic regression

In [53]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
lr_clf.score(test_features, test_labels)

0.6923076923076923

Establishing the relation labels

In [ ]:
labels = list(set([r['type'] for doc in docs for r in doc['relations']]))